In [288]:
#import the required files
import pandas as pd
import os 
import openpyxl
import re
import numpy as np

In [326]:
def complete_dataframe(df,new_filename,drawing_index,drawing_type,dp_number,drawing_id,weight,
                      dt_nomcount,dt_nomtolerance,dt_roundcount,dt_roundtolerance):

    df['Result']= new_filename # assign the Result Column to respective new value of filename
    df['Drawing_Index']= drawing_index # assign the Drawing Index Column to respective filename part
    df['Drawing_Type']= drawing_type # assign the Drawing Type Column to respective to respective filename part
    df['DP_Number']=dp_number # assign the DP Number Column to respective to respective filename part
    
    # check for the /n and /t chrachter in the drawing id
    df['Drawing_Id']= drawing_id # assign the drawing id 
    df['Weight'] = weight# assign the weight
    
    # Dimensional property assigned
    df['Dimension_Type_Nominal_Count']=dt_nomcount
    df['Dimension_Type_Nominal_Tolerance']=dt_nomtolerance
    df['Dimension_Type_Round_Count']=dt_roundcount
    df['Dimension_Type_Round_Tolerance']=dt_roundtolerance
    
#   just print statements for checking remove after use
    print(df)
    # call ouput csv function     

In [327]:
def get_drawing_info(directory):
    
    # define the required variables     
    drawing_id =""
    weight =""
    drawing_id_candidates=[]
    weights=[]
#     excel_files = []

#     # read the required directory and append in the excel file list in each and every page directory
#     for root, dirs, files in os.walk(directory):
#         for file in files:
#             if file.endswith('.xls') or file.endswith('.xlsx'):
#                 excel_files.append(os.path.join(root, file))
    
    excel_files = excel_path(directory)
    # read the files appended in excel file and fetch the drawing id & weight
    for excel_file in excel_files:
        df = pd.read_excel(excel_file,sheet_name = 'TitleBlocks') # read the sheet "Title Blockc"
        df = df.rename(columns={'Unnamed: 5':'Drawing Id Value'}) # rename the F column to "Drawing Id Value"
        
        if(pd.notnull(df.loc[3,'Drawing Id Value'])): # check for the value if it is null  or not 
            drawing_id_candidates.append(df.loc[3,'Drawing Id Value']) # append all the drawing ids in all the excel files
        
        if(pd.notnull(df.iloc[3,16])):
            weights.append(df.iloc[3,16])

    # check the length of the list    
    if len(drawing_id_candidates) > 0:
            # Use the first value as the candidate drawing_id
            drawing_id = drawing_id_candidates[0]

            # If there are more than one candidates, use the first one
            if len(drawing_id_candidates) > 1:
                drawing_id = drawing_id_candidates[0]
    # if list length = 0 add blank
    else:
        drawing_id =""    
        
    if len(weights) == 0:
        weight = ""
    elif len(set(weights)) == 1:
        weight = weights[0]
    else:
        raise AssertionError('More than one non-unique numerical value found for Weight')     
    return drawing_id,weight;

In [328]:
def excel_path(directory):
    excel_files = []
    # read the required directory and append in the excel file list in each and every page directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.xls') or file.endswith('.xlsx'):
                excel_files.append(os.path.join(root, file))
    return excel_files; 

In [329]:
def get_shape_count(directory):
    round_count=0;
    nominal_count=0;
    excel_files = excel_path(directory)
    
    # read the files appended in excel file and fetch the nominal & round count
    for excel_file in excel_files:
        df = pd.read_excel(excel_file,sheet_name = 'Measures') # read the sheet "Measures"
        shape_values = df.iloc[:, 5] # fetch data from F column
        nominal_count += shape_values.str.count("NOMINAL", flags=re.IGNORECASE).sum()
        round_count += shape_values.str.count("ROUND",flags=re.IGNORECASE).sum()
        
    return nominal_count,round_count;
    
    

In [379]:
def get_shape_tolerance(directory):
    nominal_tolerance_values=[];
    round_tolerance_values=[];
    nud=[]
    nld=[]
    rud=[]
    rld=[]
    excel_files = excel_path(directory);
    # read the files appended in excel file and fetch the nominal & round count
    for excel_file in excel_files:
        df = pd.read_excel(excel_file,sheet_name = 'Measures') # read the sheet "Measures"
#         print(df.columns)
        df['Unnamed: 12'] = df['Unnamed: 12'].astype(str)
        nominal_tolerance_values.extend(df.loc[df['Unnamed: 5'] == 'NOMINAL', 'Unnamed: 12'].values) # fetch data from F column
        round_tolerance_values.extend(df.loc[df['Unnamed: 5'] == 'ROUND', 'Unnamed: 12'].values) 
        nud.extend(df.loc[df['Unnamed: 5'] == 'NOMINAL', 'Unnamed: 9'].astype(float).values)
        nld.extend(df.loc[df['Unnamed: 5'] == 'NOMINAL', 'Unnamed: 10'].astype(float).values)
        rud.extend(df.loc[df['Unnamed: 5'] == 'ROUND', 'Unnamed: 9'].astype(float).values)
        rld.extend(df.loc[df['Unnamed: 5'] == 'ROUND', 'Unnamed: 10'].astype(float).values)

#     print(nominal_tolerance_values)
    return calculate_tolerance_value(nominal_tolerance_values,round_tolerance_values,nud,nld,rud,rld);
    
    
    

In [394]:
def calculate_tolerance_value(nominal_tvl,round_tvl,nud,nld,rud,rld):
#     print(nud,nld,rud,rld)
   
    nom_it_values = [int(val[2:]) for val in nominal_tvl if val.startswith('IT') and val[2:].isdigit()]
    nom_min_it_value = 'IT' + str(min(nom_it_values)) if nom_it_values else str(min([r - l for r, l in zip(nud or [0], nld or [0])]))
    nom_min_it_value=nom_min_it_value.replace('nan','')
    round_it_values = [int(val[2:]) for val in round_tvl if val.startswith('IT') and val[2:].isdigit()]
    round_min_it_value = 'IT' + str(min(round_it_values)) if round_it_values else str(min([r - l for r, l in zip(rud or [0], rld or [0])]))
    round_min_it_value=round_min_it_value.replace('nan','')
    return nom_min_it_value, round_min_it_value
    
    

In [332]:
# function to process names and values on directory level 
def process_file(directory):
    
    # defined the required variable for use
    new_filename=[]
    drawing_index=[]
    drawing_type=[]
    dp_number=[]
    drawing_id=[]
    weight=[]
    dt_nomcount=[]
    dt_nomtolerance=[]
    dt_roundcount=[]
    dt_roundtolerance=[]
   
    
    #created 13 columns as per mentioned in the sample csv output#
    df= pd.DataFrame(columns=['Result', 'Drawing_Index', 'Drawing_Type', 'DP_Number','Drawing_Id','Weight',
                              'Dimension_Type_Nominal_Count','Dimension_Type_Nominal_Tolerance',
                              'Dimension_Type_Round_Count','Dimension_Type_Round_Tolerance','View_Count',
                              'Inclined_Drilling_Count','Inclined_Drilling_Values'])

    
    # iterate over the directory length
    for filename in os.listdir(directory):
        
        # call get_drawing_info function by passing the filename one by one and append returned values 
        d_id,wt = get_drawing_info(directory+"/"+filename)
        drawing_id.append(d_id)
        weight.append(wt)
        
        
        # get shape count function call
        dt_nc,dt_rc = get_shape_count(directory+"/"+filename)
        dt_nomcount.append(dt_nc)
        dt_roundcount.append(dt_rc)
        
        # get shape tolerance function call
        dt_nst,dt_rst=get_shape_tolerance(directory+"/"+filename)
        dt_nomtolerance.append(dt_nst)
        dt_roundtolerance.append(dt_rst)
        
        
        
        # remove the .pdf-Results from the directorey name
        filename = filename.replace(".pdf-Results","")
        new_filename.append(filename)
        
        # split filename on '_' for finding index, type & number
        file_part= filename.split('_')
        
        # condition for asigning the specific split parts to respective index
        if len(file_part)==5:
            drawing_index.append(file_part[1])
            drawing_type.append(file_part[3])
            dp_number.append(file_part[4])
        else:
            drawing_index.append("")
            drawing_type.append("")
            dp_number.append("")
    # complete dataframe function called and updated
    complete_dataframe(df,new_filename,drawing_index,drawing_type,dp_number,drawing_id,weight,
                       dt_nomcount,dt_nomtolerance,dt_roundcount,dt_roundtolerance)     

In [395]:
def main():
    # directory path mentioned
    # try to ad dynamic path parsing
    directory = 'D:\My doc\ImportantDoc\Task\Challenge_and_Instructions_Workstudent_Data_Analyst\Challenge_and_Instructions_Workstudent_Data_Analyst\Results_Task_edited'
    process_file(directory)
        
if __name__=='__main__':
    main()

                   Result Drawing_Index Drawing_Type DP_Number  \
0    Res0001_AC_4_ETZ_001            AC          ETZ       001   
1    Res0002_AC_0_RTZ_001            AC          RTZ       001   
2    Res0003_AL_6_RTZ_001            AL          RTZ       001   
3    Res0004_AB_4_GEZ_001            AB          GEZ       001   
4    Res0005_AG_0_RTZ_001            AG          RTZ       001   
5    Res0006_AC_4_ETZ_001            AC          ETZ       001   
6    Res0007_AF_0_RTZ_001            AF          RTZ       001   
7    Res0008_AB_0_RTZ_001            AB          RTZ       001   
8    Res0009_AD_3_RTZ_001            AD          RTZ       001   
9    Res0010_AC_0_RTZ_001            AC          RTZ       001   
10   Res0011_AC_0_RTZ_001            AC          RTZ       001   
11   Res0012_AD_0_RTZ_001            AD          RTZ       001   
12   Res0013_AC_2_ETZ_001            AC          ETZ       001   
13   Res0014_AF_0_RTZ_001            AF          RTZ       001   
14   Res00